In [1]:
import pandas as pd
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import numpy as np
from sklearn.utils import shuffle

In [2]:
text_len = pd.read_csv('../test_perceptron/after_shuffle_data.csv',header=None)
text_len[:5]
# text = shuffle(text_len)
# text
# text.to_csv('test_perceptron/after_shuffle_data.csv',header=False,index=False)

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,...,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558
0,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,?,?,?,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
2,26,122,4.6923,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
3,62,62,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
4,80,87,1.0875,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.


In [3]:
Lr_label = list(set(text_len[1558].values))
Lr_label
                  

['nonad.', 'ad.']

In [17]:

# Lr_label = ['nonad.','ad.']
Lr_label = sorted(list(set(text_len[1558].values)),reverse=True)
# print(Lr_label)
def prediction(x):
    tp = []
    for w in W:
        tp.append(x.dot(w.T))
    predict_num = tp.index(max(tp))
    return predict_num

folds_number = int(len(text_len)*0.1)+1
# print(folds_number)
folds = list(range(1))

num_of_epoch = 5

for fold in folds:
#     W_dic = {}
    step = (num_of_epoch*(len(text_len)-folds_number))
    total_step = step
#     print(step,len(text_len)-folds_number,folds_number)
    W = [np.zeros(1554) for i in range(len(Lr_label))]  # W
    a = [np.zeros(1554) for i in range(len(Lr_label))]  # Averaged W
    for epoch in range(num_of_epoch):
        number_of_training = 0
        predict_number_list = list(range(folds_number * fold, folds_number * (fold + 1)))
        for text in pd.read_csv('../test_perceptron/after_shuffle_data.csv',header=None,chunksize=folds_number):
#             label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
#             text = pd.merge(text,label[[1,2]],left_on=104317,right_on=1)
            

            y = [Lr_label.index(it) for it in text[1558].values]
   
            X = text.values[:,4:-1]
            for each_data in range(len(X)):
                if number_of_training not in predict_number_list:
#                     print(number_of_training)
                    number_of_training+=1
                    y_ = prediction(X[each_data])
                    true_y = y[each_data]
#                     print(y_,true_y)
                    step -=1
                    if y_ != true_y:
                        W[true_y] = W[true_y] + X[each_data]
                        W[y_] = W[y_] - X[each_data] 
                        a[true_y] = a[true_y] + (X[each_data])*(step/(num_of_epoch*len(text_len)-folds_number))
                        a[y_] = a[y_] - (X[each_data])*(step/(num_of_epoch*len(text_len)-folds_number))
                        print('*'*(100-int(100*step/total_step))+'-'*int((step/total_step)*100)+' '+str(int(100-(step/total_step)*100)),end='\r',flush=True)
#                         print(step,len(text_len))
#                         y_ = prediction(X[each_data])
#                     if step%100 ==0:
#                         print(true_y,y_)
#                         step +=1
                else:
                    number_of_training+=1
    W = a
    number_of_training = 0
    total_y_ = []
    total_y = []

    total_y_label = [Lr_label[i] for i in total_y]
    total_y_label_ = [Lr_label[i] for i in total_y_]
    predict_number_list = list(range(folds_number * fold, folds_number * (fold + 1)))
    for text in pd.read_csv('../test_perceptron/after_shuffle_data.csv',header=None,chunksize=folds_number):
        y = [Lr_label.index(it) for it in text[1558].values]

        X = text.values[:,4:-1]
        for each_data in range(len(X)):

            if number_of_training in predict_number_list and number_of_training < folds_number* (fold + 1):
                number_of_training+=1
                y_ = prediction(X[each_data])
                true_y = y[each_data]

                total_y_.append(y_)
                total_y.append(true_y)
            elif number_of_training not in predict_number_list and number_of_training < folds_number * fold:
                number_of_training+=1
            elif number_of_training not in predict_number_list and number_of_training > folds_number * (fold + 1):
                number_of_training+=1
                print('Testing is over.')
                break

        if len(total_y_) == folds_number: # Save the perfomance of result
            
            total_y_label = [Lr_label[i] for i in total_y]
            total_y_label_ = [Lr_label[i] for i in total_y_]
            print("\nPrediction score:\n%s" % metrics.classification_report(total_y_label, total_y_label_))
            result = metrics.classification_report(total_y_label, total_y_label_)
            macro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='macro'))))
            micro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='micro'))))
            with open('result_%s.csv' % fold, 'w') as file:  # Save the metrics result to file
                for line in result:
                    file.write(line)
#                 elapsed_time = time.time() - start_time
#                 file.write('Fold %s, time is %s. \n' % (fold, elapsed_time))  # Save the time cost to file
                file.write('\n macro: '+macro)
                file.write('\n micro: '+micro)
            print('Save the result.')
            break



**************************************************************************************************** 99
Prediction score:
             precision    recall  f1-score   support

        ad.       0.89      0.89      0.89        45
     nonad.       0.98      0.98      0.98       283

avg / total       0.97      0.97      0.97       328

Save the result.


In [16]:
list(map(str,macro))

['0.935610522183', '0.935610522183', '0.935610522183', 'None']